<a href="https://colab.research.google.com/github/revy1817/NLP_BG/blob/less_1_hw/HomeWork_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets transformers

     |████████████████████████████████| 365 kB 4.3 MB/s 
     |████████████████████████████████| 4.7 MB 52.0 MB/s 
     |████████████████████████████████| 115 kB 52.1 MB/s 
     |████████████████████████████████| 212 kB 50.6 MB/s 
     |████████████████████████████████| 120 kB 32.1 MB/s 
     |████████████████████████████████| 127 kB 43.4 MB/s 
     |████████████████████████████████| 6.6 MB 45.5 MB/s 


In [20]:
import pandas as pd
import numpy as np
import logging
import torch 

from datasets import load_dataset, load_metric

from transformers.trainer import logger as noisy_logger
from transformers import AutoTokenizer, BertTokenizerFast, DataCollatorWithPadding, AutoModelForSequenceClassification, BertForSequenceClassification, TrainingArguments, Trainer, T5ForConditionalGeneration, T5Tokenizer, BertTokenizer, AutoModelForQuestionAnswering

noisy_logger.setLevel(logging.WARNING)

In [3]:
dataset = load_dataset("merionum/ru_paraphraser")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-aad01005423eb70c/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [5]:
model_name = "IlyaGusev/xlm_roberta_large_headline_cause_simple"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
label_list = sorted(set(dataset['train']['class']))
labels2id = { key:id for id, key in enumerate(label_list)}
id2labels = { id:key for id, key in enumerate(label_list)}

In [ ]:
def tokenize_and_align_labels(tokenizer, labels2id):    
    def tokenize_and_align_labels_(examples):
        tokenized_inputs = tokenizer(examples['text_1'],examples['text_2'], truncation=True)

        tokenized_inputs["labels"] = [labels2id[label] for label in examples['class']]

        return tokenized_inputs
    return tokenize_and_align_labels_

In [9]:
tokenized_input = tokenize_and_align_labels(tokenizer, labels2id)(dataset['train'][20:24])
tokenized_input

{'input_ids': [[0, 18449, 1068, 1951, 169066, 59, 42136, 59392, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2, 2, 6, 144048, 145158, 184643, 18449, 28196, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2], [0, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 7329, 155333, 24863, 5, 2, 2, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 534, 11591, 149673, 5, 2], [0, 417, 14100, 695, 1200, 49, 30037, 1269, 326, 3699, 108854, 1857, 56869, 90229, 546, 209, 9561, 5, 2, 2, 417, 68642, 69, 53177, 29, 108854, 1857, 47745, 49, 14100, 695, 1200, 90229, 546, 209, 9561, 5, 2], [0, 72085, 138109, 87830, 29, 23262, 2294, 90569, 11974, 24744, 59, 161679, 8568, 5, 2, 2, 69242, 64393, 41976, 4560, 1560, 182076, 29, 29230, 23262, 49, 72085, 138109, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [10]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0])
print(tokens)

['<s>', '▁Че', 'ч', 'ня', '▁попросил', 'а', '▁национал', 'истов', '▁со', 'ставить', '▁кодекс', '▁поведения', '▁', 'кав', 'каз', 'цев', '.', '</s>', '</s>', '▁', 'Национал', 'исты', '▁помогут', '▁Че', 'чне', '▁со', 'ставить', '▁кодекс', '▁поведения', '▁', 'кав', 'каз', 'цев', '.', '</s>']


In [11]:
dataset['train'][20]

{'id': '21',
 'id_1': '3',
 'id_2': '4116',
 'text_1': 'Чечня попросила националистов составить кодекс поведения кавказцев.',
 'text_2': 'Националисты помогут Чечне составить кодекс поведения кавказцев.',
 'class': '0'}

In [12]:
tokenized_datasets = dataset.map(tokenize_and_align_labels(tokenizer, labels2id), batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))
model.config.id2label = id2labels 
model.config.label2id = labels2id 

metric = load_metric("accuracy")

In [15]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels)

In [16]:
batch_size = 16

args = TrainingArguments(
    "paraphras",
    evaluation_strategy = "epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.05,
    save_strategy='no',
    report_to='none',
)

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.728547,0.656965
2,0.756600,0.700382,0.692827
3,0.588300,0.686240,0.705301
4,0.537800,0.694904,0.713617
5,0.498800,0.729213,0.709459


TrainOutput(global_step=2260, training_loss=0.583625192557816, metrics={'train_runtime': 1633.0474, 'train_samples_per_second': 22.127, 'train_steps_per_second': 1.384, 'total_flos': 3034369670565798.0, 'train_loss': 0.583625192557816, 'epoch': 5.0})

In [20]:
example = tokenized_datasets["test"][40:80]

In [21]:
tokens = tokenizer(example['text_1'], example['text_1'], padding=True, truncation=True, return_tensors='pt')
tokens = tokens.to('cuda:0')

with torch.no_grad():
    outputs = model(**tokens)


predicted = outputs.logits.argmax(dim=-1).cpu().numpy()
classes = [id2labels[id_label] for id_label in predicted]

In [22]:
df_example = pd.DataFrame({'text_1':example['text_1'], 'text_2':example['text_2'], 'class':example['class'], 'predict':classes})
df_example

,text_1,text_2,class,predict
0,Владимир Путин освободил от должности почти 20...,Путин снял с должностей более 20 руководителей...,0,1
1,Источник: Якунин озвучил свою месячную зарплат...,Якунин назвал размер своей зарплаты,0,1
2,Apple получила патент на социальную сеть,Apple получила патент на собственную соцсеть,1,1
3,Подвесной потолок обрушился в аэропорту Стамбу...,Четверо пострадали при обрушении подвесного по...,1,1
4,Россия представляет принципиально новый танк,Вашингтон пригрозил России новыми санкциями,-1,1
5,Генсек ООН и премьер Украины обсудили минские ...,Генсек ООН и премьер-министр Украины обсудили ...,1,1
6,Результаты выборов в Великобритании укрепили фунт,Стали известны окончательные результаты выборо...,-1,1
7,Николич: Сербия не говорила о нежелании строит...,Президент Сербии опроверг свои слова об отказе...,0,1
8,СМИ сообщают об обысках в Минкульте по делу о ...,Минкультуры отчиталось перед следствием по дел...,-1,1
9,Любляна отпразднует День Победы вместе с Москвой,Путеводитель по Дню Победы: как провести 9 мая...,-1,1
